# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1682,6,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1683,7,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1684,8,20,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
X = df.drop('dayofweek', axis=1)
y = df.dayofweek
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=21)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [4]:
parameters = {
    'kernel' : ['linear', 'rbf', 'sigmoid'],
    'C' : [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma' : ['scale', 'auto'],
    'class_weight' : ['balanced', None]
}
model = SVC(probability=True, random_state=21)
clf = GridSearchCV(model, parameters, scoring='accuracy', n_jobs=-1)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)
clf.best_params_

{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

In [5]:
clf.best_score_

0.8761090458488228

In [6]:
df = pd.DataFrame(clf.cv_results_)
df

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.531018,0.006301,0.014708,0.001290,0.01,balanced,scale,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
1,0.991137,0.039502,0.077129,0.011432,0.01,balanced,scale,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
2,0.813840,0.041966,0.021166,0.003194,0.01,balanced,scale,sigmoid,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
3,0.552510,0.052722,0.015511,0.002786,0.01,balanced,auto,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
4,0.877744,0.023240,0.065107,0.009586,0.01,balanced,auto,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.646824,0.103223,0.054083,0.006064,10,None,scale,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'scal...",0.414815,0.414815,0.422222,0.423792,0.375465,0.410222,0.017767,29
68,0.301729,0.007186,0.014841,0.000456,10,None,scale,sigmoid,"{'C': 10, 'class_weight': None, 'gamma': 'scal...",0.262963,0.244444,0.237037,0.230483,0.200743,0.235134,0.020343,46
69,28.950099,3.481298,0.009829,0.000866,10,None,auto,linear,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.737037,0.711111,0.707407,0.743494,0.698885,0.719587,0.017463,7
70,0.555630,0.025401,0.047135,0.008750,10,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1


In [7]:
df.sort_values('rank_test_score', inplace=True)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
70,0.555630,0.025401,0.047135,0.008750,10,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1
64,0.532172,0.008347,0.039806,0.002113,10,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,2
58,0.547846,0.052787,0.042451,0.002113,5,None,auto,rbf,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.825926,0.811111,0.818519,0.821561,0.802974,0.816018,0.008116,3
52,0.617492,0.050366,0.055451,0.009544,5,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.802974,0.808608,0.021007,4
63,34.943285,3.634107,0.010440,0.001369,10,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0.696140,0.084323,0.021549,0.003012,5,balanced,auto,sigmoid,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.144444,0.148148,0.137037,0.126394,0.092937,0.129792,0.019869,68
65,0.552132,0.020005,0.019033,0.000376,10,balanced,auto,sigmoid,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.122222,0.140741,0.129630,0.100372,0.085502,0.115693,0.020052,69
41,0.937150,0.211685,0.026116,0.007720,1.5,balanced,auto,sigmoid,"{'C': 1.5, 'class_weight': 'balanced', 'gamma'...",0.066667,0.085185,0.081481,0.078067,0.085502,0.079380,0.006913,70
17,1.016925,0.029460,0.028319,0.003879,0.1,balanced,auto,sigmoid,"{'C': 0.1, 'class_weight': 'balanced', 'gamma'...",0.062963,0.066667,0.062963,0.059480,0.059480,0.062310,0.002678,71


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [8]:
parameters = {
    'max_depth': range(1, 50),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini']
}
tD = tree.DecisionTreeClassifier(random_state=21)
clf = GridSearchCV(tD, parameters)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)
clf.best_params_

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}

Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}

In [9]:
clf.best_score_

0.8731212997384002

In [10]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003185,0.000590,0.001674,0.000053,balanced,entropy,1,"{'class_weight': 'balanced', 'criterion': 'ent...",0.262963,0.318519,0.266667,0.323420,0.260223,0.286358,0.028376,195
1,0.003041,0.000145,0.001589,0.000115,balanced,entropy,2,"{'class_weight': 'balanced', 'criterion': 'ent...",0.366667,0.403704,0.351852,0.408922,0.397770,0.385783,0.022437,189
2,0.003039,0.000119,0.001559,0.000146,balanced,entropy,3,"{'class_weight': 'balanced', 'criterion': 'ent...",0.374074,0.311111,0.403704,0.416357,0.408922,0.382834,0.038629,191
3,0.003179,0.000156,0.001534,0.000146,balanced,entropy,4,"{'class_weight': 'balanced', 'criterion': 'ent...",0.429630,0.351852,0.477778,0.408922,0.453532,0.424343,0.042960,187
4,0.003388,0.000157,0.001464,0.000022,balanced,entropy,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.462963,0.537037,0.533333,0.475836,0.457249,0.493284,0.034758,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,0.003804,0.000084,0.001389,0.000076,None,gini,45,"{'class_weight': None, 'criterion': 'gini', 'm...",0.862963,0.874074,0.877778,0.866171,0.836431,0.863483,0.014531,71
192,0.003869,0.000106,0.001469,0.000133,None,gini,46,"{'class_weight': None, 'criterion': 'gini', 'm...",0.862963,0.874074,0.877778,0.866171,0.836431,0.863483,0.014531,71
193,0.003824,0.000180,0.001338,0.000023,None,gini,47,"{'class_weight': None, 'criterion': 'gini', 'm...",0.862963,0.874074,0.877778,0.866171,0.836431,0.863483,0.014531,71
194,0.003777,0.000086,0.001348,0.000057,None,gini,48,"{'class_weight': None, 'criterion': 'gini', 'm...",0.862963,0.874074,0.877778,0.866171,0.836431,0.863483,0.014531,71


In [11]:
df.sort_values('rank_test_score', inplace=True)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
70,0.004381,0.000134,0.001472,0.000137,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
69,0.004365,0.000093,0.001517,0.000132,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
97,0.004270,0.000126,0.001420,0.000113,balanced,gini,49,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
95,0.004211,0.000132,0.001402,0.000138,balanced,gini,47,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
94,0.004291,0.000138,0.001383,0.000076,balanced,gini,46,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,0.002937,0.000167,0.001470,0.000148,balanced,gini,3,"{'class_weight': 'balanced', 'criterion': 'gin...",0.388889,0.303704,0.403704,0.427509,0.345725,0.373906,0.044064,192
147,0.002175,0.000070,0.001385,0.000120,None,gini,1,"{'class_weight': None, 'criterion': 'gini', 'm...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
98,0.002170,0.000092,0.001305,0.000026,None,entropy,1,"{'class_weight': None, 'criterion': 'entropy',...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
49,0.002574,0.000204,0.001324,0.000048,balanced,gini,1,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.318519,0.266667,0.323420,0.260223,0.286358,0.028376,195


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [12]:
parameters = {
    'n_estimators' : [5, 10, 50, 100],
    'max_depth': range(1, 50),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini']
}
random_forest_model = RandomForestClassifier(random_state=21)
clf = GridSearchCV(random_forest_model, parameters)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)
clf.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 28,
 'n_estimators': 50}

Best params: {'class_weight': None,
 'criterion': 'gini',
 'max_depth': 28,
 'n_estimators': 50}

In [13]:
clf.best_score_

0.9042902381935839

In [14]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008441,0.000371,0.002313,0.000231,balanced,entropy,1,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.262963,0.292593,0.225926,0.282528,0.289963,0.270794,0.024718,784
1,0.013332,0.000093,0.002716,0.000163,balanced,entropy,1,10,"{'class_weight': 'balanced', 'criterion': 'ent...",0.400000,0.414815,0.374074,0.353160,0.364312,0.381272,0.022828,776
2,0.056727,0.000471,0.006664,0.000235,balanced,entropy,1,50,"{'class_weight': 'balanced', 'criterion': 'ent...",0.511111,0.392593,0.388889,0.405204,0.453532,0.430266,0.046555,765
3,0.110693,0.000313,0.011385,0.000241,balanced,entropy,1,100,"{'class_weight': 'balanced', 'criterion': 'ent...",0.477778,0.500000,0.407407,0.420074,0.453532,0.451758,0.034578,758
4,0.008205,0.000161,0.002155,0.000062,balanced,entropy,2,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.318519,0.366667,0.381481,0.353160,0.345725,0.353110,0.021165,781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,0.152391,0.001929,0.013974,0.000082,None,gini,48,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.914815,0.911111,0.900000,0.903346,0.884758,0.902806,0.010460,5
780,0.009726,0.000169,0.002346,0.000062,None,gini,49,5,"{'class_weight': None, 'criterion': 'gini', 'm...",0.885185,0.855556,0.881481,0.877323,0.847584,0.869426,0.015004,502
781,0.017127,0.000189,0.002873,0.000043,None,gini,49,10,"{'class_weight': None, 'criterion': 'gini', 'm...",0.896296,0.859259,0.907407,0.888476,0.858736,0.882035,0.019749,362
782,0.076966,0.000250,0.007863,0.000216,None,gini,49,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.903704,0.907063,0.877323,0.902063,0.014494,39


In [15]:
df.sort_values('rank_test_score', inplace=True)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
698,0.080433,0.002714,0.008003,0.000402,None,gini,28,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,1
711,0.158228,0.007389,0.014365,0.000724,None,gini,31,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,2
314,0.082959,0.004371,0.008082,0.000291,balanced,gini,30,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.881481,0.907063,0.895911,0.902817,0.013554,3
330,0.081520,0.002782,0.008057,0.000279,balanced,gini,34,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.892593,0.907063,0.884758,0.902809,0.013010,4
702,0.083967,0.000337,0.008312,0.000184,None,gini,29,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.896296,0.911111,0.903346,0.884758,0.902806,0.011698,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,0.007557,0.000274,0.002107,0.000135,None,entropy,1,5,"{'class_weight': None, 'criterion': 'entropy',...",0.355556,0.366667,0.374074,0.345725,0.327138,0.353832,0.016467,780
4,0.008205,0.000161,0.002155,0.000062,balanced,entropy,2,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.318519,0.366667,0.381481,0.353160,0.345725,0.353110,0.021165,781
200,0.008955,0.000148,0.002433,0.000156,balanced,gini,2,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.311111,0.377778,0.377778,0.353160,0.312268,0.346419,0.029749,782
196,0.010482,0.001487,0.002587,0.000281,balanced,gini,1,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.292593,0.285185,0.282528,0.293680,0.283390,0.011062,783


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [16]:
import pip
def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])
install('tqdm')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting tqdm

Downloading tqdm-4.64.1-py2.py3-none-any.whl (78 kB)

/opt/goinfre/einterdi/Piscine_Python_Data_Science/venv/lib/python3.7/site-packages/pip/_vendor/rich/live.py:229: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Installing collected packages: tqdm

Successfully installed tqdm-4.64.1

In [17]:
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score

In [18]:
parameters = {
    'n_estimators' : [5, 10, 50, 100],
    'max_depth': range(1, 50),
    'class_weight' : ['balanced', None],
    'criterion' : ['entropy', 'gini']
}
grid = list(ParameterGrid(parameters))

frame = []
for i in tqdm(grid):
    random_forest_model.set_params(**i)
    scores = cross_val_score(random_forest_model, X_train, y_train, cv=5, n_jobs=-1)
    i['mean_accuracy'] = np.mean(scores)
    i['std_accuracy'] = np.std(scores)
    frame.append(i)

100%|██████████| 784/784 [01:25<00:00,  9.15it/s]


In [19]:
df = pd.DataFrame(frame).sort_values('mean_accuracy', ascending=False)
df

,class_weight,criterion,max_depth,n_estimators,mean_accuracy,std_accuracy
698,None,gini,28,50,0.904290,0.010961
711,None,gini,31,100,0.903547,0.014380
314,balanced,gini,30,50,0.902817,0.013554
330,balanced,gini,34,50,0.902809,0.013010
751,None,gini,41,100,0.902806,0.010460
...,...,...,...,...,...,...
392,None,entropy,1,5,0.353832,0.016467
4,balanced,entropy,2,5,0.353110,0.021165
200,balanced,gini,2,5,0.346419,0.029749
196,balanced,gini,1,5,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [20]:
random_forest_model = RandomForestClassifier(random_state=21, n_estimators=50, max_depth=28, criterion='gini')
random_forest_model.fit(X_train, y_train)
y_after_test = random_forest_model.predict(X_test)
accuracy_score(y_test, y_after_test)

0.9289940828402367